# Pandas: Union Tables

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('./../data/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('./../data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#### Сколько строк в файле рейтингов, не считая строки заголовка:

In [4]:
ratings.count()

userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64

#### Какое количество жанров имеют фильмы в датасете movies?

Каждый фильм может относиться как к одному, так и к нескольким жанрам верно

#### Какое минимальное значение принимает выставленная оценка в датасете ratings?

In [5]:
ratings['rating'].min()

0.5

#### Какое максимальное значение принимает выставленная оценка в датасете ratings?

In [6]:
ratings['rating'].max()

5.0

## Полезные ссылки
- Наглядный [пример](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) различных режимов склейки таблиц по строкам или столбцам (метод concat). Пригодится, чтобы быстро вспомнить, как изменять типы объединения таблиц.
- [Документация метода merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) — пригодится, если забылись названия основных параметров (по каким столбцам объединяем таблицы и каким способом).
- [Объяснение](http://www.skillz.ru/dev/php/article-Obyasnenie_SQL_obedinenii_JOIN_INNER_OUTER.html) типов объединений таблиц — если возникнут трудности с выбором типа объединения (в pandas.merge параметр how).

#### Сколько фильмов в таблице movies?

In [7]:
movies.count()

movieId    9742
title      9742
genres     9742
dtype: int64

## Объединяем таблицы

In [8]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Схематично метод merge можно описать так: joined = left_df.merge(right_df, on='', how='').

Давайте разберем подробнее параметры метода: 

- **left_df** / **right_df** — датафреймы, которые мы объединяем. К "правому" датафрейму присоединяем "левый" (в нашем примере "левый" датафрейм — ratings, "правый" — movies). 
- **how** — параметр объединения записей. Он может иметь четыре значения: *left, right, inner и outer*.
  - *left* берем все записи (movieId) из "левого" датафрейма (ratings) и ищем их соответствия в "правом" (movies). В итоговом датафрейме останутся только те значения, которым были найдены соответствия, то есть только значения из ratings.
  - *right* остаются только значения из "правого" датафрейма. Если совпадений между таблицами нет, то ставим нулевое значение.
  - *inner* оставляет только те записи (movieId), которые есть в обоих датафреймах.
  - *outer* объединяет все варианты movieId в обоих датафреймах. 
- **on** определяет, по какому столбцу происходит объединение. Для объединения по нескольким столбцам используйте on = ['col1', 'col2'] или left_on и right_on.


После объединения датафреймов лучше проверять, что не возникло дубликатов. О возможных проблемах метода merge мы поговорим в следующем блоке. Сейчас убедимся в том, что число строк объединенного датафрейма совпадает с исходным:

In [9]:
len(ratings) == len(joined)

True

Получаем значение True — значит, число строк совпадает.

## Трудности объединения датафреймов

In [10]:
ratings_small = pd.read_csv('./../data/ratings_example.txt', sep = '\t')
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [11]:
movies_small = pd.read_csv('./../data/movies_example.txt', sep = '\t')
movies_small.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


#### В текущей версии датасета movies значение movieId = 31 встречается дважды, а movieId = 32 - один раз. Определите, при каких типах объединения датафреймов ratings и movies по столбцу movieId в итоговом датафрейме останутся оба значения movieId, при условии использования следующего синтаксиса:

ratings_small.merge(movies_small, on = 'movieId', how = ...)

In [12]:
ratings_small.merge(movies_small, on='movieId', how='right')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
2,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [13]:
ratings_small.merge(movies_small, on='movieId', how='outer')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
2,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


## Pandas: Дубликаты строк

In [14]:
ratings_small.merge(movies_small, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


### Удаляем дубликаты
Если вы хотите избежать подобной ситуации, необходимо удалить дубликаты из таблицы movies. Для этого подходит метод **drop_duplicates**. В параметре subset указываем один или несколько столбцов, по комбинации которых хотим удалить дубликаты.

С помощью параметра **keep** указываем, какой из встречающихся дубликатов оставить (например, первый или последний). Параметр **inplace** указывает, что изменения нужно сохранить в датафрейме, к которому применяется метод (в нашем случае — в датафрейме movies):

In [17]:
movies_small_uniq = movies_small.drop_duplicates(subset='movieId', keep='first', inplace=False)
movies_small_uniq.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


Теперь объединение таблиц будет корректным:

In [18]:
ratings_small.merge(movies_small_uniq, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


#### При каком типе объединения таблиц с помощью метода merge (т. е. при каком значении параметра how) не могут возникать дубликаты строк? В качестве примера можете использовать объединение датафреймов ratings_small и movies_small из этого шага:

в любом типе могут быть дубликаты верно

In [23]:
ratings_small.merge(movies_small, how='outer', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
2,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
